In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2

2024-07-01 23:56:00.035858: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 23:56:00.215777: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 23:56:01.065939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 23:56:07.408962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_inicial = pd.read_csv('/home/darkcover/Documentos/Out/dados/data_final2.csv')
data_inicial = data_inicial.drop(columns=['Unnamed: 0'])
print("Data Carregada ...")

Data Carregada ...


In [3]:
# Verificar se há valores NaN ou infinitos
print(np.isnan(data_inicial).sum())
print(np.isinf(data_inicial).sum())

Rodada               0
level                0
apostar              0
acerto               0
contagem             0
odd                  0
odd_entrada          0
odd_saida            0
media80              0
desvpad80geral       0
percentil80geral     0
cv80                 0
roc80                1
media160             0
desvpad160geral      0
percentil160geral    0
cv160                0
roc160               1
media320             0
desvpad320geral      0
percentil320geral    0
cv320                0
roc320               1
media640             0
desvpad640geral      0
percentil640geral    0
cv640                0
roc640               1
dtype: int64
Rodada               0
level                0
apostar              0
acerto               0
contagem             0
odd                  0
odd_entrada          0
odd_saida            0
media80              0
desvpad80geral       0
percentil80geral     0
cv80                 0
roc80                0
media160             0
desvpad160geral      

In [4]:
data_inicial = data_inicial.dropna().reset_index(drop=True)

In [5]:
X1 = data_inicial[['odd_entrada', "media80", "desvpad80geral", "percentil80geral", "cv80", "roc80", "media160", "desvpad160geral", "percentil160geral", "cv160", "roc160","media320", "desvpad320geral", "percentil320geral", "cv320", "roc320", "media640", "desvpad640geral", "percentil640geral", "cv640", "roc640"]].values

y1 = data_inicial['odd_saida'].values

In [6]:
# Verificar se há valores NaN ou infinitos
print(np.isnan(X1).sum())
print(np.isinf(X1).sum())

0
0


In [7]:
# Normalizar os recursos (importante para redes neurais)
scaler1 = StandardScaler()
X1 = scaler1.fit_transform(X1)

# Dividir os dados em treinamento e teste
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

# Ajustar os rótulos para binarizar em relação a 5
y_train_bin = binarize(y_train1.reshape(-1, 1), threshold=5).reshape(-1)
y_test_bin = binarize(y_test1.reshape(-1, 1), threshold=5).reshape(-1)

In [10]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

# Criação dos modelos individuais
rf_model = RandomForestClassifier(n_estimators=100)
gb_model = GradientBoostingClassifier(n_estimators=100)
lr_model = LogisticRegression()

# Treinamento dos modelos individuais (exceto LSTM, que já foi treinado)
rf_model.fit(X_train1, y_train_bin)
gb_model.fit(X_train1, y_train_bin)
lr_model.fit(X_train1, y_train_bin)

# Criação do ensemble
ensemble_model = VotingClassifier(estimators=[ 
    ('rf', rf_model), 
    ('gb', gb_model), 
    ('lr', lr_model)], voting='soft')

ensemble_model.fit(X_train1, y_train_bin)
predictions_ensemble = ensemble_model.predict(X_test1)


In [14]:
# Definir o modelo com a nova camada de saída
model_binary = Sequential()
model_binary.add(Input(shape=(X_train1.shape[1], 1)))
model_binary.add(LSTM(64, return_sequences=True))
model_binary.add(Dropout(0.2))
model_binary.add(LSTM(64))
model_binary.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model_binary.add(Dense(1, activation='sigmoid'))  # Saída binária para probabilidade

# Compilar o modelo com ajustes
model_binary.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Treinar o modelo com os rótulos binarizados
model_binary.fit(X_train1.reshape(X_train1.shape[0], X_train1.shape[1], 1), y_train_bin, epochs=5, batch_size=640, validation_split=0.2)

# Resumo do modelo
model_binary.summary()

# Avaliar o modelo no conjunto de teste
accuracy_binary = model_binary.evaluate(X_test1.reshape(X_test1.shape[0], X_test1.shape[1], 1), y_test_bin)[1]
print(f'Acurácia do modelo binário: {accuracy_binary}')

# Fazer previsões com probabilidade
predictions_binary = model_binary.predict(X_test1.reshape(X_test1.shape[0], X_test1.shape[1], 1))

# Arredondar as previsões para obter rótulos de classe
predicted_labels_binary = (predictions_binary >= 0.5).astype(int)

# Calcular o F1-Score para a nova configuração binária
f1_binary = f1_score(y_test_bin, predicted_labels_binary, average='weighted')

print(f'F1-Score do modelo binário: {f1_binary}')

Epoch 1/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 101s 24ms/step - accuracy: 0.5780 - loss: 0.6883 - val_accuracy: 0.5756 - val_loss: 0.6818
Epoch 2/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 143s 24ms/step - accuracy: 0.5755 - loss: 0.6819 - val_accuracy: 0.5756 - val_loss: 0.6817
Epoch 3/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 151s 26ms/step - accuracy: 0.5763 - loss: 0.6822 - val_accuracy: 0.5756 - val_loss: 0.6820
Epoch 4/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 135s 25ms/step - accuracy: 0.5796 - loss: 0.6807 - val_accuracy: 0.5756 - val_loss: 0.6817
Epoch 5/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 143s 25ms/step - accuracy: 0.5799 - loss: 0.6805 - val_accuracy: 0.5756 - val_loss: 0.6817
Epoch 6/50
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 100s 25ms/step - accuracy: 0.5765 - loss: 0.6815 - val_accuracy: 0.5756 - val_loss: 0.6817
Epoch 7/50
 706/4000 ━━━━━━━━━━━━━━━━━━━━ 1:25 26ms/step - accuracy: 0.5773 - loss: 0.6812

KeyboardInterrupt: 

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# Criar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo
rf_model.fit(X_train1, y_train_bin)

# Fazer previsões
rf_predictions = rf_model.predict(X_test1)

# Calcular a acurácia e o F1-Score
rf_accuracy = accuracy_score(y_test_bin, rf_predictions)
rf_f1 = f1_score(y_test_bin, rf_predictions, average='weighted')

print(f'Acurácia do Random Forest: {rf_accuracy}')
print(f'F1-Score do Random Forest: {rf_f1}')

Acurácia do Random Forest: 0.540975
F1-Score do Random Forest: 0.5199612104668544


In [10]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

# Criar o modelo XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=24,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    random_state=42
)

# Treinar o modelo
xgb_model.fit(X_train1, y_train_bin)

# Fazer previsões
xgb_predictions = xgb_model.predict(X_test1)

# Calcular a acurácia e o F1-Score
xgb_accuracy = accuracy_score(y_test_bin, xgb_predictions)
xgb_f1 = f1_score(y_test_bin, xgb_predictions, average='weighted')

print(f'Acurácia do XGBoost: {xgb_accuracy}')
print(f'F1-Score do XGBoost: {xgb_f1}')


Acurácia do XGBoost: 0.537625
F1-Score do XGBoost: 0.5179303167980445


In [21]:
from sklearn.linear_model import LogisticRegression

# Criar o modelo de regressão logística
logreg_model = LogisticRegression(random_state=42)

# Treinar o modelo
logreg_model.fit(X_train1, y_train_bin)

# Fazer previsões
logreg_predictions = logreg_model.predict(X_test1)

# Calcular a acurácia e o F1-Score
logreg_accuracy = accuracy_score(y_test_bin, logreg_predictions)
logreg_f1 = f1_score(y_test_bin, logreg_predictions, average='weighted')

print(f'Acurácia da Regressão Logística: {logreg_accuracy}')
print(f'F1-Score da Regressão Logística: {logreg_f1}')


Acurácia da Regressão Logística: 0.579025
F1-Score da Regressão Logística: 0.425071370331179


In [ ]:
from sklearn.svm import SVC

# Criar o modelo SVM
svm_model = SVC(probability=True, random_state=42)

# Treinar o modelo
svm_model.fit(X_train1, y_train_bin)

# Fazer previsões
svm_predictions = svm_model.predict(X_test1)

# Calcular a acurácia e o F1-Score
svm_accuracy = accuracy_score(y_test_bin, svm_predictions)
svm_f1 = f1_score(y_test_bin, svm_predictions, average='weighted')

print(f'Acurácia do SVM: {svm_accuracy}')
print(f'F1-Score do SVM: {svm_f1}')
